In [1]:
import os
import sys

os.environ['SPARK_HOME'] = "D:\program\spark2.4.2"
os.environ['PYSPARK_PYTHON'] = "D:\program\python3\python3.exe"
os.environ['HADOOP_HOME'] = "D:\program\hadoop-common-2.7.1-bin"

sys.path.append("D:\program\spark2.4.2\bin")
sys.path.append("D:programspark2.4.2\python")
sys.path.append("D:programspark2.4.2\python\pyspark")
sys.path.append("D:programspark2.4.2\python\lib")
sys.path.append("D:programspark2.4.2\python\lib\pyspark.zip")
sys.path.append("D:programspark2.4.2\python\lib\py4j-0.10.4-src.zip")
sys.path.append("D:\program\java8")

from pyspark import SparkContext
from pyspark import SparkConf


sc = SparkContext("local","testing")

In [2]:
print(sc)

<SparkContext master=local appName=testing>


In [7]:
user_data = sc.textFile(r"E:\dataset\ml-20m\ratings_noheader.csv") #需要进行数据清洗，数据补全，不然后面会出错

In [8]:
user_data.first()

'1,2,3.5,1112486027'

In [9]:
rates = user_data.map(lambda x: x.split(",")[0:3])
print (rates.first())

['1', '2', '3.5']


In [12]:
from pyspark.mllib.recommendation import Rating
rates_data = rates.map(lambda x: Rating(int(x[0]),int(x[1]),float(x[2])))
print (rates_data.first())

Rating(user=1, product=2, rating=3.5)


In [13]:
from  pyspark.mllib.recommendation import ALS
from pyspark.mllib.recommendation import MatrixFactorizationModel
sc.setCheckpointDir('checkpoint/')
ALS.checkpointInterval = 2
model = ALS.train(ratings=rates_data, rank=20, iterations=5, lambda_=0.02) #这个步骤老是出现java.io.IOException: (null) entry in command string: null chmod 0644这种问题，需要添加合适的hadoop_home变量

In [16]:
print (model.predict(1,112)) #预测用户对摸个产品的评分

3.297501876796062


In [20]:
print (model.recommendUsers(253,10))#将商品推荐给10个最合适的用户，可以看到用户对这个商品的评价都超过5分了，满分才5分。。。。

[Rating(user=56659, product=253, rating=7.021295905408631), Rating(user=123554, product=253, rating=6.805486120699479), Rating(user=71549, product=253, rating=6.642656716768418), Rating(user=103796, product=253, rating=6.5748299661609115), Rating(user=110749, product=253, rating=6.255862925709703), Rating(user=81103, product=253, rating=6.107264678510025), Rating(user=68658, product=253, rating=6.083039343870528), Rating(user=71934, product=253, rating=6.079418299685771), Rating(user=42482, product=253, rating=6.069459149662893), Rating(user=9886, product=253, rating=5.988796806115877)]


In [19]:
print(model.recommendProducts(1,10)) #推荐10个商品

[Rating(user=1, product=104390, rating=6.993582348535744), Rating(user=1, product=73529, rating=6.518953358851498), Rating(user=1, product=97300, rating=6.501910406515705), Rating(user=1, product=56779, rating=5.98893669429358), Rating(user=1, product=104583, rating=5.913042434398518), Rating(user=1, product=62206, rating=5.773360929900679), Rating(user=1, product=69464, rating=5.715772436339162), Rating(user=1, product=26033, rating=5.68207954792214), Rating(user=1, product=73194, rating=5.665705419399374), Rating(user=1, product=73139, rating=5.596993193337324)]


In [25]:
#print(model.recommendProductsForUsers(5).collect())#给所有用户推荐5条产品，这个计算量很大，不要随便算

spark = sc.getOrCreate()
spark.createDataFrame(rates_data)

AttributeError: 'SparkContext' object has no attribute 'createDataFrame'

In [23]:
print(rates_data.show(5))

AttributeError: 'PipelinedRDD' object has no attribute 'show'